## Basic Testing

In [1]:
import numpy as np
dataset = np.load('./LondonAQ.npz')
dataset

In [2]:
ls = dataset.files
ls

['GEltham', 'GWesthorne', 'BSladeGreen', 'GWoolwich']

In [3]:
dataset[ls[0]].shape

(43848, 9)

In [4]:
dataset[ls[0]][0]

array([12.4,  0. ,  5.7, 54.4,  2.8,  1. ,  1. ,  6. ,  0. ])

## Convert dataset to input

In [5]:
vars = [0, 1, 2, 3, 4]
airq = {}
for d in ls:
        airq[d] = dataset[d]
        if vars is not None:
            airq[d] = airq[d][:, vars]

In [6]:
data = airq[ls[0]][:, 0].reshape(-1, 1)  ## We only use the first one feature from 9 features here.
data.shape

(43848, 1)

In [7]:
datasize = 35064
testsize = 8784
lag = 6
ahead = 1

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [9]:
wind_train = data[:datasize, :]
wind_train.shape

(35064, 1)

### Executing below codes and trying to print train[0], train[1] and train[2]. Finding something from them!

In [10]:
ahead = 1 - 1
lvect = []
for i in range(lag):
    lvect.append(wind_train[i: -lag - ahead + i])
lvect.append(wind_train[lag + ahead:])
train = np.stack(lvect, axis=1)
train.shape

(35058, 7, 1)

In [11]:
train_x, train_y = train[:, :lag], train[:, -1:, 0]

In [12]:
wind_test = data[datasize:datasize + testsize, 0].reshape(-1, 1)
lvect = []
for i in range(lag):
    lvect.append(wind_test[i: -lag - ahead + i])
lvect.append(wind_test[lag + ahead:])
test = np.stack(lvect, axis=1)
test_x, test_y = test[:, :lag], test[:, -1:, 0]

### Now we get the data for our RNN: train_x, train_y, test_x, test_y

### Build RNN Model

In [14]:
def architecture(neurons, drop, nlayers, activation, activation_r, rnntype, impl=1):
    """
    RNN architecture

    :return:
    """
    RNN = LSTM if rnntype == 'LSTM' else GRU
    model = Sequential()
    if nlayers == 1:
        model.add(RNN(neurons, input_shape=(train_x.shape[1], train_x.shape[2]), implementation=impl,
                      recurrent_dropout=drop, activation=activation, recurrent_activation=activation_r))
    else:
        model.add(RNN(neurons, input_shape=(train_x.shape[1], train_x.shape[2]), implementation=impl,
                      recurrent_dropout=drop, activation=activation, recurrent_activation=activation_r,
                      return_sequences=True))
        for i in range(1, nlayers - 1):
            model.add(RNN(neurons, recurrent_dropout=drop, implementation=impl,
                          activation=activation, recurrent_activation=activation_r, return_sequences=True))
        model.add(RNN(neurons, recurrent_dropout=drop, activation=activation,
                      recurrent_activation=activation_r, implementation=impl))

    model.add(Dense(1))

    return model

### Load information from config.json

In [15]:
import json
def load_config_file(nfile, abspath=False):
    """
    Read the configuration from a json file

    :param abspath:
    :param nfile:
    :return:
    """
    ext = '.json' if 'json' not in nfile else ''
    pre = '' if abspath else './'
    fp = open(pre + nfile + ext, 'r')

    s = ''

    for l in fp:
        s += l

    return json.loads(s)

config = load_config_file('config')

In [20]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM, GRU

impl = 2
model = architecture(neurons=config['arch']['neurons'],
                         drop=config['arch']['drop'],
                         nlayers=config['arch']['nlayers'],
                         activation=config['arch']['activation'],
                         activation_r=config['arch']['activation_r'], rnntype=config['arch']['rnn'], impl=impl)

In [21]:
optimizer = config['training']['optimizer']

if optimizer == 'rmsprop':
    if 'lrate' in config['training']:
        optimizer = RMSprop(lr=config['training']['lrate'])
    else:
        optimizer = RMSprop(lr=0.001)

model.compile(loss='mean_squared_error', optimizer=optimizer)

In [22]:
cbacks = []
model.fit(train_x, train_y, batch_size=config['training']['batch'],
          epochs=config['training']['epochs'],
          validation_data=(test_x, test_y),
          verbose=False, callbacks=cbacks)